In [1]:
years = """
<div class="filter__column column-1">
<select class="filter__select-dropdown select-dropdown background-color--white arrow-box--background-color--greyscale-15" name="filter_season_id" id="filter_season_id">
<option value="0">Change Season</option><option value="24">2024</option>
<option value="23" selected="selected">2023</option>
<option value="22">2022</option>
<option value="21">2021</option>
<option value="10">2020</option>
<option value="9">2019</option>
<option value="8">2018</option>
<option value="7">2017</option>
<option value="6">2016</option>
<option value="5">2015</option>
<option value="1">2014</option>
<option value="2">2013</option>
<option value="3">2012</option>
<option value="4">2011</option>
<option value="11">2010</option>
<option value="12">2009</option>
<option value="13">2008</option>
<option value="14">2007</option>
<option value="15">2006</option>
<option value="16">2005</option>
<option value="17">2004</option>
<option value="18">2003</option>
<option value="19">2002</option>
<option value="20">2001</option>
</select>
</div>"""

race_traks = """
<div class="filter__column column-2">
<select id="filter_meeting_id" class="filter__select-dropdown select-dropdown background-color--white arrow-box--background-color--greyscale-15" name="filter_meeting_id">
<option value="0">Meeting</option><option value="211" selected="selected">Monza</option>
<option value="197">Brands Hatch</option>
<option value="200">Circuit Paul Ricard 1000Km</option>
<option value="204">CrowdStrike 24 Hours of Spa</option>
<option value="203">Misano</option>
<option value="213">Nürburgring </option>
<option value="205">Hockenheim</option>
<option value="206">Valencia</option>
<option value="207">Barcelona</option>
<option value="201">Zandvoort</option>
</select>
</div>
"""

race = """
<div class="filter__column column-3">
<select id="filter_race_id" class="filter__select-dropdown select-dropdown background-color--white arrow-box--background-color--greyscale-15" name="filter_race_id">
<option value="">Session</option>
<option value="1378">Race 2</option>
<option value="1377">Qualifying 2</option>
<option value="1376">Race 1</option>
<option value="1375">Qualifying 1</option>
<option value="1374">Pre Qualifying</option>
<option value="1373">Free practice</option>
<option value="1372">Official Paid Test Sessions 1</option>
</select> 
</div>
"""



In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_race_data(filter_season_id, filter_meeting_id, filter_race_id):
    # Словари для хранения соответствий ID
    seasons = {
        2024: "24",
        2023: "23",
        2022: "22",
        # Добавьте остальные необходимые соответствия
    }
    meetings = {
        "Barcelona": "207",
        "Misano": "203",
        # Добавьте остальные необходимые соответствия
    }
    
    # Проверяем, есть ли указанные ID в словарях
    season_value = seasons.get(filter_season_id)
    meeting_value = meetings.get(filter_meeting_id)
    
    if season_value is None or meeting_value is None:
        raise ValueError("Указаны неверные значения для сезона или встречи.")
    
    # URL для первого запроса (для получения списка сессий)
    url = f'https://www.gt-world-challenge-europe.com/results?filter_season_id={season_value}&filter_meeting_id={meeting_value}&filter_race_id='
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Поиск нужного filter_race_id по тексту ссылки
    race_options = soup.select('#filter_race_id option')
    race_value = None
    for option in race_options:
        if option.text.strip() == filter_race_id:
            race_value = option['value']
            break
    
    if race_value is None:
        raise ValueError(f"Сессия {filter_race_id} не найдена.")
    
    # URL для второго запроса (для получения данных сессии)
    final_url = f'https://www.gt-world-challenge-europe.com/results?filter_season_id={season_value}&filter_meeting_id={meeting_value}&filter_race_id={race_value}'
    
    response = requests.get(final_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Парсинг данных сессии
    rows = soup.select('.table__body tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        data.append([col.text.strip() for col in cols])
    
    # Создание DataFrame
    df_columns = ['Pos', 'Car #', 'Class', 'Drivers', 'Team', 'Car', 'Time', 'Laps', 'Gap']
    df = pd.DataFrame(data, columns=df_columns)
    
    return df
